# Importaciones y funciones necesarias

In [ ]:
#importamos la librerias necesarias
from numpy import nan
import pandas as pd
import ast

In [ ]:
#creamos una funcion para buscar el indice name 
def buscar_nombre(lista):
    #en este caso esto convierte los objetos a listas
    lista = ast.literal_eval(lista)
    # en caso de haber mas de un diccionario con la clave name devuelve todos los valores concatenados
    return ', '.join([i['name'] for i in lista])

#creamos otra funcion con la misma logica  que buscar nombre pero modificada para funcionar con diccionarios
def buscar_nombre_diccionario(dicc):
    #en este caso esto convierte los objetos a diccionarios
    dicc = ast.literal_eval(dicc)
    return dicc['name']

#creamos una funcion para buscar el indice name si el job es igual a director
def buscar_nombre_director(lista):
    #en este caso esto convierte los objetos a listas
    lista = ast.literal_eval(lista)
    for i in lista:
            if i['job'] == 'Director':
                return i['name']
            else: continue


# ETL de movies_dataset.csv

In [ ]:
#leemos el csv
movies = pd.read_csv('./data/movies_dataset.csv')
#comprobamos que el dataframe este cargado correctamente
movies

In [ ]:
#comprobamos el tamaño del dataframe
print(movies.shape)
#removemos los valores duplicados
movies.drop_duplicates(inplace=True)
print(movies.shape)
#removemos las columnas que no se utilizaran y/o estan duplicadas
movies = movies.drop(columns=['video', 'adult', 'imdb_id', 'original_title', 'poster_path', 'homepage', 'overview','tagline'])
print(movies.shape)


In [ ]:
#remplazamos los nulos por 0
movies['revenue'] = movies['revenue'].fillna('0')
movies['budget'] = movies['budget'].fillna('0')

In [ ]:
#removemos las filas con valores faltantes en la columna release_date
movies = movies.dropna(subset=['release_date'])
#eliminamos las filas que no sigan el formato AAAA-MM-DD en la columna release_date
movies['release_date'] = pd.to_datetime(movies['release_date'], format='%Y-%m-%d', errors='coerce')
movies = movies.dropna(subset=['release_date'])

In [ ]:
#creamos la columna year 
movies['released_year'] = movies['release_date'].dt.year

In [ ]:
#cambiamos el tipo de las columnas budget y revenue a int para poder hacer operaciones
movies['budget'] = movies['budget'].astype(int)
movies['revenue'] = movies['revenue'].astype(int)
#creamos la columna return que es la columna revenue entre la columna budget
    #en caso de que uno de los numeros sea 0 devolvera 0
movies['return'] = movies.apply(lambda fila: fila['revenue'] / fila['budget'] if fila['budget'] != 0 else 0, axis=1)


In [ ]:
#aplicamos la funcion buscar_nombre a las columnas para desanidar los datos 
movies['genres'] = movies['genres'].fillna('[]').apply(buscar_nombre)
movies['spoken_languages'] = movies['spoken_languages'].fillna('[]').apply(buscar_nombre)
movies['production_companies'] = movies['production_companies'].fillna('[]').apply(buscar_nombre)
movies['production_countries'] = movies['production_countries'].fillna('[]').apply(buscar_nombre)

In [ ]:
#en caso de no haber datos en belongs_to_collection la funcion fillna crea un diccionario con la clave name y el valor "NA"
    #de no hacerlo daria un key error al ejecutar la funcion buscar_nombre_diccionario
movies['belongs_to_collection'] = movies['belongs_to_collection'].fillna('{"name":"NA"}').apply(buscar_nombre_diccionario)

movies['belongs_to_collection'] = movies['belongs_to_collection'].replace('NA', nan)

In [ ]:
movies

# etl de credits.csv

In [ ]:
#leemos el csv
credits = pd.read_csv('./data/credits.csv')
#comprobamos que el dataset se haya cargado correctamente
credits

In [ ]:
#comprobamos el tamaño del dataframe
print(credits.shape)
# eliminamos los duplicados y volvemos a comprobar el tamaño del dataframe
credits.drop_duplicates(inplace=True)
print(credits.shape)

In [ ]:
# obtenemos los nombres de los actores
credits['cast'] = credits['cast'].fillna('[]').apply(buscar_nombre)

In [ ]:
#obtenemos los nombres de los directores
credits['director'] = credits['crew'].apply(buscar_nombre_director)

In [ ]:
#eliminamos la columna crew para reducir significativamente el peso del dataframe
credits = credits.drop(columns='crew')

In [ ]:
credits

# concatenamos y exportamos como csv

In [ ]:
movies = pd.concat([movies, credits], axis=1, join='inner')
movies = movies.drop(columns='id.1')
movies_limpio_path = 'data/movies_limpio.csv'

movies.to_csv(movies_limpio_path, sep=',', index=False, encoding='utf-8')
